<a href="https://colab.research.google.com/github/valerio-unifei/ECAA07/blob/main/ECAA07_TR1_Modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho 1 - SQL

Nome: Carlos Henrique Valério de Moraes

Matrícula: (professor)

# Banco Original em Arquivo CSV

## Analisando Arquivo CSV

In [ ]:
import pandas as pd

banco = pd.read_csv('https://github.com/valerio-unifei/ECAA07/raw/main/Bancos/ecaa07inv_079.csv')
banco.head()

In [ ]:
#convertendo campo numérico para data
banco['data'] = pd.to_datetime(banco['data'], format='%Y%m%d')
# ordenando por id do fundo e data do rendimento
banco = banco.sort_values(by = ['id','data'])
banco.head()

In [ ]:
# informações sobre a base
print(banco.info())
print(banco.nunique())

## Criando Arquivo SQLite com Tabela Auxiliar

In [ ]:
!rm ecaa07inv_079.db

In [ ]:
import sqlite3
with sqlite3.connect('ecaa07inv_079.db') as conn:
  banco.to_sql('auxiliar',conn)

# Banco de Dados em SQLite

In [ ]:
%load_ext sql

%sql sqlite:///ecaa07inv_079.db

## Criando Tabelas do Banco

In [ ]:
%%sql 
CREATE TABLE IF NOT EXISTS classes (
  classe_id   INTEGER PRIMARY KEY AUTOINCREMENT,
  classe      TEXT NOT NULL
);

In [ ]:
%%sql 
CREATE TABLE IF NOT EXISTS fundos (
  fundo_id    INTEGER PRIMARY KEY AUTOINCREMENT,
  cnpj        TEXT,
  id          INTEGER NOT NULL,
  fundo       TEXT NOT NULL,
  classe_id   INTEGER REFERENCES classes(classe_id) ON DELETE CASCADE,
  minimo      REAL,
  taxa        INTEGER,
  estudo      INTEGER,
  fator       REAL
);

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS rendimentos (
  rendimento_id INTEGER PRIMARY KEY AUTOINCREMENT,
  fundo_id      INTEGER REFERENCES fundos(fundo_id) ON DELETE CASCADE,
  rendimento    REAL,
  data          DATETIME,
  patrimonio    REAL,
  calculo       REAL,
  momento       INTEGER,
  custo         REAL
);

### Verificando estrutura

In [ ]:
%sql SELECT * FROM sqlite_master

# Inserindo Dados

## Usando Tabela Auxiliar na Importação

In [ ]:
%sql SELECT * FROM auxiliar LIMIT 10

In [ ]:
%sql INSERT INTO classes (classe) SELECT DISTINCT classe FROM auxiliar

In [ ]:
%%sql
  INSERT INTO fundos (cnpj, id, fundo, minimo, taxa, estudo, fator, classe_id)
  SELECT DISTINCT a.cnpj, a.id, a.fundo, a.minimo, a.taxa, a.estudo, a.fator, b.classe_id
  FROM auxiliar as a INNER JOIN classes as b ON a.classe = b.classe

In [ ]:
%%sql
  INSERT INTO rendimentos (fundo_id, rendimento, data, patrimonio, calculo, momento, custo)
  SELECT DISTINCT a.fundo_id, b.rendimento, b.data, b.patrimonio, b.calculo, b.momento, b.custo
  FROM fundos as a INNER JOIN auxiliar as b ON a.id = b.id

In [ ]:
%sql DROP TABLE IF EXISTS auxiliar

# Criando consultas

## Ajustando SQL

In [ ]:
%%sql 
  SELECT b.cnpj, b.id, a.classe, b.fundo, b.minimo, b.taxa, b.estudo, b.fator, c.rendimento, c.data, c.patrimonio, c.calculo, c.momento, c.custo
  FROM classes as a INNER JOIN fundos as b ON a.classe_id = b.classe_id
    INNER JOIN rendimentos as c ON b.fundo_id = c.fundo_id
  ORDER BY b.id, c.data
  LIMIT 10

In [ ]:
%%sql 
SELECT b.fundo, (MAX(c.patrimonio) - MIN(c.patrimonio))/(MAX(JULIANDAY(c.data)) - MIN(JULIANDAY(c.data))) as diariomedio
    FROM fundos as b INNER JOIN rendimentos as c ON b.fundo_id = c.fundo_id
    GROUP BY b.fundo
    ORDER BY diariomedio DESC

## Criando VIEWS

In [ ]:
%%sql 
CREATE VIEW IF NOT EXISTS original as
    SELECT b.cnpj, b.id, a.classe, b.fundo, b.minimo, b.taxa, b.estudo, b.fator, c.rendimento, c.data, c.patrimonio, c.calculo, c.momento, c.custo
    FROM classes as a INNER JOIN fundos as b ON a.classe_id = b.classe_id
      INNER JOIN rendimentos as c ON b.fundo_id = c.fundo_id;

In [ ]:
%%sql 
CREATE VIEW IF NOT EXISTS melhor_fundo as
  SELECT *

In [ ]:
# descobrir como implementar
%%sql
  SELECT * from original

### Testando Views

In [ ]:
%sql SELECT * FROM original

In [ ]:
%sql SELECT * FROM melhor_fundo